In [3]:
import pandas as pd
import numpy as np

# 索引与pandas的数据运算

相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。自动的数据对齐操作在不重叠的索引处引入了NA值。缺失值会在算术运算过程中传播。

**外连接**

In [5]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [6]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [7]:
s1+s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

对于DataFrame，对齐操作会同时发生在行和列上

In [8]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [9]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [10]:
# 把它们相加后将会返回一个新的DataFrame，其索引和列为原来那两个DataFrame的并集
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


如果DataFrame对象相加，没有共用的列或行标签，结果都会是空(全NaN)

In [12]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})
df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


In [13]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),columns=list('abcd'))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [14]:
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),columns=list('abcde'))
df2.loc[1,"b"] = np.nan
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [15]:
# 将它们相加时，没有重叠的位置就会产生NA值
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


使用df1的add方法，传入df2以及一个fill_value参数

In [16]:
df1.add(df2, fill_value=0)


,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


# DataFrame和Series之间的运算

In [19]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [20]:
# series与frame的第0行匹配
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着行一直向下广播

In [21]:
# series是frame的某一行，运算时按行向下广播
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集

In [23]:
ser2 = pd.Series(range(3), index=['b', 'e', 'f'])
ser2

b    0
e    1
f    2
dtype: int64

In [24]:
frame - ser2

,b,d,e,f
Utah,0.0,NaN,1.0,NaN
Ohio,3.0,NaN,4.0,NaN
Texas,6.0,NaN,7.0,NaN
Oregon,9.0,NaN,10.0,NaN


上述都是逐行运算——如果你希望匹配行且在列上广播，则必须使用**算术运算方法**

In [27]:
series = frame.loc[:,"d"]
series

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [29]:
frame.sub(series, axis="index")# 使用index指定按列运算

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


In [30]:
#也可以使用asix=0来指定
frame.sub(series, axis=0)

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


# 函数运用

## NumPy的ufuncs（元素级数组方法）也可用于操作pandas对象

In [31]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.075019,1.574032,1.993682
Ohio,-1.699905,-1.324265,0.293401
Texas,1.855294,-0.352802,-0.109860
Oregon,-0.244483,-1.553297,-0.126439


In [33]:
np.abs(frame)

,b,d,e
Utah,0.075019,1.574032,1.993682
Ohio,1.699905,1.324265,0.293401
Texas,1.855294,0.352802,0.109860
Oregon,0.244483,1.553297,0.126439


## apply-Series级（行/列数据）

将函数应用到由各列或行所形成的一维数组上。DataFrame的apply方法即可实现此功能

In [34]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [37]:
f = lambda x : x.max() - x.min()
# 函数f，计算了一个Series的最大值和最小值的差，在frame的每列都执行了一次
frame.apply(f)

b    9.0
d    9.0
e    9.0
dtype: float64

In [38]:
# 传递axis=’columns’到apply，这个函数会在每行执行
frame.apply(f, axis="columns")


Utah      2.0
Ohio      2.0
Texas     2.0
Oregon    2.0
dtype: float64

传递到apply的函数不是必须返回一个标量，还可以返回由多个值组成的Series

In [39]:
def f(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])
frame.apply(f)

,b,d,e
min,0.0,1.0,2.0
max,9.0,10.0,11.0


## applymap、map-元素级

### dataframe-applymap

In [51]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [42]:
# 使用函数将frame中所有数据浮点数进行格式化
format = lambda x : "%.2f" % x
frame.applymap(format)

,b,d,e
Utah,0.00,1.00,2.00
Ohio,3.00,4.00,5.00
Texas,6.00,7.00,8.00
Oregon,9.00,10.00,11.00


### series-map

In [46]:
format = lambda x : "%.2f" % x
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.map(format)
                

d    0.00
a    1.00
b    2.00
c    3.00
dtype: object

In [60]:
frame.loc["Ohio",:].map(format)

b    3.00
d    4.00
e    5.00
Name: Ohio, dtype: object

# 排序和排名

## 索引排序

In [61]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [62]:

obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [63]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),index=['three', 'one'],columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [66]:
# 默认按轴0-行索引进行排序
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


## 数据值排序

sort_values

In [71]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame    

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项

In [74]:
# 对b列的所有数据进行降序排序
frame.sort_values(by="b",axis=0, ascending=False)

,b,a
1,7,1
0,4,0
3,2,1
2,-3,0


In [75]:
# 要根据多个列进行排序，传入名称的列表即可
frame.sort_values(by=["a", "b"])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1
